## Generating multiple Bash scripts

This notebook is an example of a notebook which generates either a single or multiple bash scripts to run Sorcha. If you want to use this, **you will need to heavily edit this to suit your own particular setup.**

This notebook does assume that you have a local copy of the input files on the machine where you are running this notebook. If you don't, you'll have to edit this a bit more.

In the case presented below, there are 16 folders each containing ~100 sets of input files for Sorcha. An example of a folder layout and file/folder naming system is shown below.

![alternative text](example_file_structure.png)

In [ ]:
import glob
import os

First get sorted lists of all of the orbit and physical parameters files.

In [ ]:
orbits_all = glob.glob('./*/*orbit.s3m')

In [ ]:
orbits_all.sort()

In [ ]:
len(orbits_all)

In [ ]:
params_all = glob.glob('./*/*physical.s3m')

In [ ]:
params_all.sort()

In [ ]:
len(params_all)

In this example, we are running on a workstation with 32 cores, so:

In [ ]:
1572/32

We need fifty runs across 32 cores. So we can set up a script to run on one core that contains fifty **consecutive** Sorcha runs, each overwriting the same ephemeris file. We can define our Sorcha input/outputs like so:

In [ ]:
script_filepath = './sorcha_runscript.sh'
config_filepath = './config_file.ini'
pointing_filepath = './baseline_v3.0_10yrs.db'
ephemeris_filepath = './ephemeris_file_core_1.csv'
output_filepath = './outputs/'
cache_filepath = './sorcha_cache/'

The below will create this single runscript.

In [ ]:
with open(script_filepath, "a") as the_file:
        the_file.write("#!/bin/bash\n")

for orbit_file in orbits_all[:50]:

    stem_filepath = orbit_file.split('_')[0]
    stem_identifier = orbit_file.split('_')[1]

    physical_file = '_'.join([stem_filepath, stem_identifier, 'physical.s3m'])

    output_file = '_'.join([stem_filepath.split('/')[-1], stem_identifier, 'output.csv'])
    
    sorcha_command = ('sorcha -c '
        + config_filepath
        + ' -p '
        + physical_file
        + ' -ob '
        + orbit_file
        + ' -e '
        + pointing_filepath
        + ' -o '
        + output_filepath
        + ' -t '
        + output_file
        + '-ar'
        + cache_filepath)

    with open(script_filepath, "a") as the_file:
        the_file.write(sorcha_command + " \n")

To generate all 32 runscripts, one for each core, run this (making sure to change script_filepath).

In [ ]:
for i in range(0, 33):

    # CHANGE THESE IF YOU WANT THEM TO BE DIFFERENT. 
    # The other locations (pointing database, config filename) will be pulled from previous cell.
    script_filepath = './sorcha_DP03_runscript_'+str(i)+'.sh'

    with open(script_filepath, "a") as the_file:
        the_file.write("#!/bin/bash\n")
    
    for orbit_file in orbits_all[i*50:(i*50)+50]:
    
        stem_filepath = orbit_file.split('_')[0]
        stem_identifier = orbit_file.split('_')[1]
    
        physical_file = '_'.join([stem_filepath, stem_identifier, 'physical.s3m'])
    
        output_file = '_'.join([stem_filepath.split('/')[-1], stem_identifier, 'output.csv'])
        
        sorcha_command = ('sorcha -c '
            + config_filepath
            + ' -p '
            + physical_file
            + ' -ob '
            + orbit_file
            + ' -e '
            + pointing_filepath
            + ' -o '
            + output_filepath
            + ' -t '
            + output_file        
            + '-ar'
            + cache_filepath)
    
        with open(script_filepath, "a") as the_file:
            the_file.write(sorcha_command + " \n")
